## 导入数据，数据清洗

载入包文件

In [2]:
import pandas as pd
import numpy as np
import json
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"
# %load_ext scalene

载入数据

In [3]:
# 载入电影数据
data_item = pd.read_csv('../douban_movie/data/movies.csv')
print('电影数目：' ,data_item.shape[0])

电影数目： 2060


In [4]:
# 总评分最高的前10部电影
data_item.sort_values('rate', ascending=False)[['title','rate']].head(10)

,title,rate
1582,陈奕迅线上慈善演唱会 Live Is So Much Better With Music E...,9.7
1765,岚ARAFES2020at国立竞技场 ARASHI ARAFES 2020 at Natio...,9.6
1841,This is 嵐 LIVE,9.6
1071,宇多田光Laugher in the Dark 2018 巡回演唱会 Hikaru Utad...,9.5
1223,星野源巡回演唱会：流行病毒 GEN HOSHINO STADIUM TOUR “POP VI...,9.5
1919,山河令生来知己演唱会,9.5
1497,安德烈·波切利米兰大教堂空场独唱 Andrea Bocelli : Music for Ho...,9.4
1770,Funtastic Babii Fanmeeting,9.4
1807,TME Live 吴青峰「16叶」线上演唱会 TME Live 吳青峰「16葉」線上演唱會,9.4
1662,鱼丁糸Follow Me线上演唱会,9.4


In [5]:
def column_expand(data, column, list_values):
    for cl in list_values:
        tt = data_item[column].apply(lambda x: str(x)).str.contains('\W'+cl+'\W')
        uu = data_item[column].apply(lambda x: str(x)).str.contains('^'+cl+'$')
        ee = data_item[column].apply(lambda x: str(x)).str.contains(cl+'\s')
        ff = data_item[column].apply(lambda x: str(x)).str.contains('\s'+cl)
#         print('cl', cl)
#         print('tt', tt)
#         print('uu', uu)
#         print('ee', ee)
        cl_ = tt | uu| ee | ff
        cl_ *= 1
#         print('c1_', cl_)
        data['%s_%s' %(column ,cl)] = cl_
#         print('data', data)
        
class_movie = ['剧情','爱情','喜剧','科幻','动作','悬疑','犯罪','恐怖','青春'
               ,'励志','战争','文艺','黑色幽默','传记','情色','暴力','音乐','家庭']
country_movie = ['大陆','美国','香港','台湾','日本','韩国','英国','法国','德国','意大利','西班牙','印度','泰国','俄罗斯'
 ,'伊朗','加拿大','澳大利亚','爱尔兰','瑞典','巴西','丹麦']

def get_values_list(data, column, sep=None):
    Language_values=[]
    def countLANG(Languages):
        for language in Languages:
            language = language.strip()
            if language in Language_values: 
                continue
            else:
                Language_values.append(language)
    if sep:
        pd.DataFrame(data[column].str.split(sep))[column].apply(countLANG);
    else:
        data[column].apply(countLANG);
    return Language_values

def Paiming(data, column, list_values):
    column_expand(data, column, list_values)
    df = pd.DataFrame( 
        {'数目':[data['%s_%s' %(column, p)].sum() for p in list_values]}
        , index=list_values).sort_values('数目', ascending=False)
    return df

In [6]:
# 最受欢迎的电影类别排名
movie_type_list = Paiming(data_item, 'type', class_movie)
print(movie_type_list)
# 根据结果显示发现在2019-2021年三年的时间里爱情、科幻、悬疑、动作、喜剧 类型的电影，其中最受欢迎的是爱情题材的电影

       数目
爱情    139
科幻    138
悬疑    132
动作    129
喜剧    121
恐怖    108
剧情     68
音乐     44
家庭     44
传记     22
战争     17
犯罪     17
情色     11
青春      0
文艺      0
黑色幽默    0
暴力      0
励志      0


# 词云分析

In [7]:
import warnings
warnings.filterwarnings("ignore")
import jieba   # 分词包
import numpy
import codecs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import matplotlib
matplotlib.rc('figure', figsize = (14, 7))
matplotlib.rc('font', size = 14)
matplotlib.rc('axes', grid = False)
matplotlib.rc('axes', facecolor = 'white')
from wordcloud import WordCloud # 词云包

In [8]:
def cut_word(content):
    # 导入、分词
    segment=[]
    for line in content: 
        try:
            segs = jieba.lcut(line) # jiaba.lcut() 
#             print(segs)
            for seg in segs:
                if len(seg)>1 and seg!='\r\n':
                    segment.append(seg)
        except:
            print(line)
            continue
#     print('cut_word', segment[:1000])
    return segment
cut_word(['阔别三十多年的同学，生活在不同的地方，有才华横溢事业有成的，有投资有方商海弄潮的，也有固守田园风光难舍故乡,真的好'])

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/j9/x9wd0pm15rgd97b0jsp8sk100000gn/T/jieba.cache
Loading model cost 0.352 seconds.
Prefix dict has been built successfully.


['阔别',
 '三十多年',
 '同学',
 '生活',
 '不同',
 '地方',
 '才华横溢',
 '事业有成',
 '投资',
 '有方',
 '商海',
 '弄潮',
 '固守',
 '田园风光',
 '难舍',
 '故乡',
 '真的']

In [9]:
def remove_stopword(segment):
    # 去停用词
    words_df = pd.DataFrame({'segment':segment})
    stopwords = pd.read_csv("../stopwords.txt" 
                          ,index_col=False
                          ,quoting=3
                          ,sep="\t"
                          ,names=['stopword']
                          ,encoding='utf-8') # quoting=3 全不引用    
    words_df=words_df[~words_df.segment.isin(stopwords.stopword)]
    return words_df

In [10]:
# 统计词频
def get_word_count(words_df):
    words_stat = words_df.groupby('segment').agg(计数=pd.NamedAgg(column='segment', aggfunc='size')).reset_index().sort_values(by='计数', ascending=False)
    return words_stat

In [11]:
from __future__ import division, print_function
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
import seaborn as sns
def createHist(x_data):
#     ax1.hist(x_data, histtype='stepfilled', bins=200, color = y1_color)
    plt.bar(x_width,y_data,lw=0.5,fc="r",width=0.3,label="Phone")
    ax1.set_title(title)
    plt.ylabel('Number')
    plt.xlabel('High frequency words of each movie')

In [12]:
# 词云展示
import imageio
matplotlib.rcParams['figure.figsize']=(10.0,10.0)
from wordcloud import WordCloud,ImageColorGenerator
def createWordCloud(imgPath, words_stat={}):
    bimg=imageio.imread('cover.jpg')
    wordcloud=WordCloud(background_color="white",mask=bimg,font_path='../simhei.ttf',max_font_size=200)
    word_frequence={x[0]:x[1] for x in words_stat.head(1000).values}
    wordcloud=wordcloud.fit_words(word_frequence)
    bimgColors=ImageColorGenerator(bimg)
    plt.axis("off")
    plt.imshow(wordcloud.recolor(color_func=bimgColors))

In [13]:
def createWordCloudByGenre(content, imgPath='cover.jpg'):
    segment = cut_word(content)
#     print('segment', segment)
#     segment = tf_idf_words(segment)
#     print('segment', segment)
    words_df = remove_stopword(segment)
    words_stat = get_word_count(words_df)
    createWordCloud(imgPath,words_stat=words_stat)

In [14]:
# 读取影评数据集
data_com = pd.read_csv('../douban_movie/data/comment.csv')

In [15]:
# 合并电影短评
movie_comment_data = pd.merge(data_item, data_com, on="movie_id")

In [16]:
# 对最受欢迎的前五条电影类别进行分析
movie_top_five = movie_type_list.head(5).index
# movie_type={x: x.index for x in movie_type_list}
print(movie_top_five)
# # 获取方式某个题材的影评
def get_comment(type):
    flag = movie_comment_data['type'].apply(lambda x: str(x)).str.contains(type)
    content = movie_comment_data[flag].content
    return content

Index(['爱情', '科幻', '悬疑', '动作', '喜剧'], dtype='object')


In [17]:
comment = get_comment('爱情').values.tolist()

In [18]:
segment = cut_word(comment)
words_df = remove_stopword(segment)
words_stat = get_word_count(words_df).head(10)
print(words_stat)

nan
      segment    计数
38113      故事  4301
22459      喜欢  2864
28648      导演  2595
26113      女主  1784
46471      演员  1745
46536      演技  1662
49637      男主  1650
9542       不错  1490
26679      好看  1454
59741      角色  1431


In [19]:
# 词云展示最受欢迎的五个类型评论

# createWordCloudByGenre(comment)

In [20]:
# comment = get_comment('科幻').values.tolist()
# segment = cut_word(comment)
# words_df = remove_stopword(segment)
# words_stat = get_word_count(words_df).head(10)
# print(words_stat)
# createWordCloudByGenre(comment)

In [21]:
# comment = get_comment('悬疑').values.tolist()
# segment = cut_word(comment)
# words_df = remove_stopword(segment)
# words_stat = get_word_count(words_df).head(10)
# print(words_stat)
# createWordCloudByGenre(comment)

In [22]:
# comment = get_comment('动作').values.tolist()
# createWordCloudByGenre(comment)

In [23]:
# comment = get_comment('喜剧').values.tolist()
# segment = cut_word(comment)
# words_df = remove_stopword(segment)
# words_stat = get_word_count(words_df).head(10)
# print(words_stat)
# createWordCloudByGenre(comment)

## 2.情感分析

In [24]:
# from snownlp import SnowNLP
# # 读取评论数据
# df = pd.read_csv('../douban_movie/data/comment.csv')
# df = df.dropna(subset=['content'])
# df['segments'] = df['content'].apply(lambda x: (SnowNLP(x).sentiments))
# # df['segments'] = df['content'].apply(lambda x: print(x))
# # print(df['segments'])
# # print(SnowNLP('asdad').sentiments)

In [25]:
# df.to_csv('comment-segments.csv')

In [26]:
df = pd.read_csv('../douban_movie/data/comment-segments.csv')
df = pd.merge(data_item, df, on="movie_id")
# print(len(df['id']))
def getRate(df,type):
    positive = ['力荐', '推荐']
    middle = ['还行']
    negative = ['较差', '很差']
    praise_flag = df['degree'].apply(lambda x: x in positive)
    middle_flag = df['degree'].apply(lambda x: x in middle)
    negative_flag = df['degree'].apply(lambda x: x in negative)
    number_of_praise = len(df[praise_flag])
    number_of_middle = len(df[middle_flag])
    number_of_negative = len(df[negative_flag])
    total = len(df['id'])
    print('-----------{type}---------'.format(type=type))
    print('total', total)
    print('好评', number_of_praise)
    print('差评', number_of_negative)
    print('中评', number_of_middle)
    print('好评率', number_of_praise/total)
    print('差评率', number_of_negative/total)
    print('中评率', number_of_middle/total)

In [54]:
import jieba.posseg as pseg
import jieba.analyse
try:
    words = pd.read_csv('./data/data3.csv')
except FileNotFoundError:
    arr = df.content
    data_set = []
    for content in arr:
        segment = []
    #     jieba.analyse.set_stop_words('../customer-stopword.txt')
        segs = jieba.analyse.extract_tags(content, topK=20, allowPOS=('n', 'v'), withWeight=False)
#         segs = pseg.cut(content)
        stopwords = pd.read_csv("../stopwords.txt" 
                                  ,index_col=False
                                  ,quoting=3
                                  ,sep="\t"
                                  ,names=['stopword']
                                  ,encoding='utf-8') # quoting=3 全不引用 
        for seg in segs:
            if  len(seg)>1 and seg!='\r\n' and seg not in stopwords.stopword.values.tolist():
                segment.append(seg)
        data_set.append(segment)
    data = pd.DataFrame(data_set)
    data.to_csv('./data/data3.csv', index=False, header=False)

In [69]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=data_set, size=50, window=5, min_count=1, workers=8)
model.save("./data/word2vec3.model")
model.wv.most_similar('创意', topn=30)

KeyboardInterrupt: 

In [76]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec.load('./data/word2vec3.model')
model.wv.most_similar('演员', topn=30)

[('用力', 0.9325686097145081),
 ('级别', 0.9115071892738342),
 ('到位', 0.9104755520820618),
 ('细节', 0.9037952423095703),
 ('手法', 0.8996878266334534),
 ('印象', 0.8994144797325134),
 ('可惜', 0.8985620141029358),
 ('美术', 0.8977164030075073),
 ('功夫', 0.8965245485305786),
 ('出色', 0.8956985473632812),
 ('灾难', 0.893528938293457),
 ('优点', 0.8924545645713806),
 ('剧本', 0.8902140259742737),
 ('观众', 0.8898880481719971),
 ('失望', 0.8860753178596497),
 ('节奏', 0.8856069445610046),
 ('版本', 0.8839591145515442),
 ('大片', 0.8838419914245605),
 ('难受', 0.8825384974479675),
 ('舞台', 0.8811585903167725),
 ('表达', 0.8803365230560303),
 ('对话', 0.8800433874130249),
 ('经典', 0.8791768550872803),
 ('戏剧', 0.8782491683959961),
 ('欣赏', 0.8781362175941467),
 ('味道', 0.8778901100158691),
 ('放到', 0.8777268528938293),
 ('广告', 0.8770118951797485),
 ('技巧', 0.8768482208251953),
 ('无力', 0.8766248822212219)]

In [62]:
model.wv.most_similar('情节', topn=30)

[('亮点', 0.9846416115760803),
 ('到位', 0.9832262992858887),
 ('缺点', 0.9822530150413513),
 ('优点', 0.9816826581954956),
 ('主线', 0.9811536073684692),
 ('灾难', 0.980776846408844),
 ('线索', 0.9770481586456299),
 ('场面', 0.9768010973930359),
 ('模仿', 0.9761210083961487),
 ('动机', 0.9752260446548462),
 ('痕迹', 0.9742719531059265),
 ('细节', 0.9735087752342224),
 ('交代', 0.9723002314567566),
 ('出色', 0.9710083603858948),
 ('基调', 0.9700902104377747),
 ('美术', 0.9694360494613647),
 ('塑造', 0.9686477780342102),
 ('影子', 0.9686455130577087),
 ('对话', 0.968528151512146),
 ('氛围', 0.9675323367118835),
 ('沦为', 0.9659230709075928),
 ('级别', 0.965461015701294),
 ('味道', 0.9653477668762207),
 ('切换', 0.9647563099861145),
 ('还原', 0.9645223021507263),
 ('无比', 0.9636674523353577),
 ('文艺', 0.9635986089706421),
 ('功夫', 0.9620676040649414),
 ('改编', 0.96144038438797),
 ('没能', 0.9607720375061035)]

In [64]:
model.wv.most_similar('导演', topn=30)

[('表演', 0.8574581742286682),
 ('出色', 0.8331863880157471),
 ('戏剧', 0.82332444190979),
 ('影片', 0.8232444524765015),
 ('团队', 0.8200812935829163),
 ('出土', 0.8178362250328064),
 ('到位', 0.8148158192634583),
 ('大片', 0.8129892945289612),
 ('体验', 0.8121553063392639),
 ('经典', 0.8102008700370789),
 ('影子', 0.80657958984375),
 ('手法', 0.8056279420852661),
 ('剧本', 0.8047341704368591),
 ('美术', 0.8041030764579773),
 ('技巧', 0.8034223318099976),
 ('灾难', 0.8012386560440063),
 ('用力', 0.8007007837295532),
 ('层面', 0.7997066974639893),
 ('观众', 0.7997002005577087),
 ('失望', 0.7994277477264404),
 ('级别', 0.7982085347175598),
 ('拍摄', 0.7977235913276672),
 ('场面', 0.7968255877494812),
 ('细节', 0.7968000769615173),
 ('情感', 0.7967529892921448),
 ('优点', 0.7962159514427185),
 ('个性', 0.7948406338691711),
 ('逻辑', 0.7942978143692017),
 ('舞台', 0.7942209839820862),
 ('缺点', 0.7930411100387573)]

In [68]:
model.wv.most_similar('镜头', topn=30)

[('调度', 0.9789342284202576),
 ('刻意', 0.9691569805145264),
 ('略显', 0.9676828384399414),
 ('野心', 0.9646788835525513),
 ('叙事', 0.9610164165496826),
 ('构图', 0.9607267379760742),
 ('剧作', 0.9596087336540222),
 ('饱满', 0.9587934017181396),
 ('摄影', 0.9587878584861755),
 ('美学', 0.9547196626663208),
 ('渲染', 0.9541066288948059),
 ('新人', 0.9540977478027344),
 ('碎片', 0.9526451230049133),
 ('衔接', 0.9522627592086792),
 ('震撼', 0.9511477947235107),
 ('沉浸', 0.9466251134872437),
 ('空洞', 0.9463822245597839),
 ('廉价', 0.9451215267181396),
 ('克制', 0.9450045824050903),
 ('打动', 0.9443309903144836),
 ('堪称', 0.9438174366950989),
 ('话剧', 0.943645179271698),
 ('手持', 0.9436127543449402),
 ('写实', 0.941469132900238),
 ('转折', 0.9406062960624695),
 ('电影节', 0.940559983253479),
 ('人物形象', 0.9381392002105713),
 ('编排', 0.9366158843040466),
 ('融入', 0.9361589550971985),
 ('松散', 0.9356771111488342)]

In [70]:
model.wv.most_similar('拍摄', topn=30)

[('痕迹', 0.981736421585083),
 ('文艺', 0.9762628674507141),
 ('视觉', 0.9761107563972473),
 ('还原', 0.9731016159057617),
 ('摄影', 0.9709812998771667),
 ('出色', 0.968325138092041),
 ('美术', 0.967509925365448),
 ('用心', 0.9657629728317261),
 ('模仿', 0.9647272825241089),
 ('切换', 0.9638059139251709),
 ('氛围', 0.9626973867416382),
 ('演绎', 0.9614977240562439),
 ('场景', 0.961313784122467),
 ('缺点', 0.9610882997512817),
 ('夸张', 0.9609147906303406),
 ('刻画', 0.960405170917511),
 ('剧作', 0.95957350730896),
 ('尺度', 0.9575209021568298),
 ('情节', 0.9574722051620483),
 ('画面', 0.9569975733757019),
 ('剧本', 0.9565473794937134),
 ('灾难', 0.9561775326728821),
 ('改编', 0.9543325304985046),
 ('沦为', 0.9538736939430237),
 ('叙述', 0.9538387656211853),
 ('塑造', 0.9528836607933044),
 ('气质', 0.951707661151886),
 ('毛病', 0.9512271881103516),
 ('魅力', 0.9500613808631897),
 ('主线', 0.9498776793479919)]

In [75]:
model.wv.most_similar('拍摄', topn=150)

[('痕迹', 0.981736421585083),
 ('文艺', 0.9762628674507141),
 ('视觉', 0.9761107563972473),
 ('还原', 0.9731016159057617),
 ('摄影', 0.9709812998771667),
 ('出色', 0.968325138092041),
 ('美术', 0.967509925365448),
 ('用心', 0.9657629728317261),
 ('模仿', 0.9647272825241089),
 ('切换', 0.9638059139251709),
 ('氛围', 0.9626973867416382),
 ('演绎', 0.9614977240562439),
 ('场景', 0.961313784122467),
 ('缺点', 0.9610882997512817),
 ('夸张', 0.9609147906303406),
 ('刻画', 0.960405170917511),
 ('剧作', 0.95957350730896),
 ('尺度', 0.9575209021568298),
 ('情节', 0.9574722051620483),
 ('画面', 0.9569975733757019),
 ('剧本', 0.9565473794937134),
 ('灾难', 0.9561775326728821),
 ('改编', 0.9543325304985046),
 ('沦为', 0.9538736939430237),
 ('叙述', 0.9538387656211853),
 ('塑造', 0.9528836607933044),
 ('气质', 0.951707661151886),
 ('毛病', 0.9512271881103516),
 ('魅力', 0.9500613808631897),
 ('主线', 0.9498776793479919),
 ('到位', 0.949418842792511),
 ('线索', 0.9479015469551086),
 ('内涵', 0.9476402997970581),
 ('体验', 0.9475505948066711),
 ('技巧', 0.9472385644912

## 3.主题分析

In [72]:
model.wv.most_similar('剧情', topn=30)

KeyError: "word '剧情' not in vocabulary"

In [ ]:
len(df.content)

In [ ]:
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import warnings
warnings.filterwarnings('ignore')  # To ignore all warnings that arise here to enhance clarity
 
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

In [ ]:
# 数据处理
content_by_type = {}
for type in movie_top_five:
    flag = df['type'].apply(lambda x: str(x)).str.contains(type)
    content = df[flag].content.values.tolist()
    content_by_type[type] = content
# print(content_by_type['爱情'][:100])

In [ ]:
import jieba.analyse
# 构建词典
data_set = []
content_X = df.content
for content in content_X:
    segment = []
#     jieba.analyse.set_stop_words('../customer-stopword.txt')
    segs = jieba.analyse.extract_tags(content, topK=20, allowPOS=('n'), withWeight=False)
#     segs = jieba.lcut(content)
    stopwords = pd.read_csv("../stopwords.txt" 
                              ,index_col=False
                              ,quoting=3
                              ,sep="\t"
                              ,names=['stopword']
                              ,encoding='utf-8') # quoting=3 全不引用 
    for seg in segs:
        if len(seg)>1 and seg!='\r\n' and seg not in stopwords.stopword.values.tolist():
            segment.append(seg)
    data_set.append(segment)
print(data_set[:100])
dictionary = corpora.Dictionary(data_set)  # 构建词典
# print(dictionary.token2id)
corpus = [dictionary.doc2bow(text) for text in data_set]
# print(corpus)

In [ ]:
# 构建LDA模型
import pprint
from gensim.models.wrappers import LdaMallet
from gensim.models import CoherenceModel
import gensim
num_topics = 8
# 传统lda词袋30
# ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30,random_state = 1)   #分为5个主题,text:文本，已经表示成词袋了。num_topics: 提取的主题数id2word:词典passes:类似于在机器学习中常见的epoch，也就是训练了多少轮;random_state -这是一个种子（如果您想精确地重复训练过程）。
# pprint.pprint(ldamodel.print_topics(num_topics=num_topics, num_words=15))  #每个主题输出5个单词

#LdaMallet模型 50
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=8, id2word=dictionary)
pprint.pprint(ldamallet.show_topics(formatted=False))

# tfidf 47
# tfidf = gensim.models.TfidfModel(corpus)
# corpus_tfidf = tfidf[corpus]
# ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30,random_state = 1)   #分为5个主题,text:文本，已经表示成词袋了。num_topics: 提取的主题数id2word:词典passes:类似于在机器学习中常见的epoch，也就是训练了多少轮;random_state -这是一个种子（如果您想精确地重复训练过程）。
# pprint.pprint(ldamodel.print_topics(num_topics=num_topics, num_words=15))  #每个主题输出5个单词


# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_set, dictionary=dictionary, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(sentences=data_set, size=50, window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [ ]:
model.wv.most_similar('摄影', topn=30)

In [ ]:
model.wv.most_similar('演员', topn=300)

In [ ]:
model.wv.most_similar('特效', topn=300)

In [ ]:
#计算困惑度
def perplexity(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=15))
    print(ldamodel.log_perplexity(corpus))
    return ldamodel.log_perplexity(corpus)
#计算coherence
def coherence(num_topics):
    ldamodel = LdaModel(corpus, num_topics=num_topics, id2word = dictionary, passes=30,random_state = 1)
    print(ldamodel.print_topics(num_topics=num_topics, num_words=10))
    ldacm = CoherenceModel(model=ldamodel, texts=data_set, dictionary=dictionary, coherence='c_v')
#     print(ldacm.get_coherence())
    return ldacm.get_coherence()

In [ ]:
# 绘制主题-coherence曲线，选择最佳主题数
x = range(1,15)
y = [perplexity(i) for i in x]  #如果想用困惑度就选这个
# y = [coherence(i) for i in x]
plt.plot(x, y)
plt.xlabel('主题数目')
plt.ylabel('coherence大小')
plt.rcParams['font.sans-serif']=['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False
plt.title('主题-coherence变化情况')
plt.show()

In [ ]:
#  通过上述主题评估，我们发现可以选择3作为主题个数，接下来我们可以再跑一次模型，设定主题数为5，并输出每个文档最有可能对应的主题
from gensim.models import LdaModel
import pandas as pd
from gensim.corpora import Dictionary
from gensim import corpora, models

 
dictionary = corpora.Dictionary(data_set)  # 构建词典
corpus = [dictionary.doc2bow(text) for text in data_set]
 
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, passes = 30,random_state=1)
topic_list=lda.print_topics()
pprint.pprint(topic_list)
for i in lda.get_document_topics(corpus)[:]:
    print('i==',i)
    listj=[]
    for j in i:
        print('j==', j)
        listj.append(j[1])
#     print(listj)
    bz=listj.index(max(listj))
#     print(i[bz][0])

# for topic in lda.print_topics(num_words = 20):
#     termNumber = topic[0]
#     print(topic[0], ':', sep='')
#     listOfTerms = topic[1].split('+')
#     for term in listOfTerms:
#         listItems = term.split('*')
#         print('  ', listItems[1], '(', listItems[0], ')', sep='')

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
# 用pyLDAvis将LDA模式可视化
plot =pyLDAvis.gensim_models.prepare(lda,corpus,dictionary)
# pyLDAvis.show(plot)
# 保存到本地html
pyLDAvis.save_html(plot, 'pyLDAvis.html')
